In [20]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'C:/a/big-query--cd85abb654f9.json' 

import dotenv
import pandas as pd
from google.cloud import bigquery
import google.auth
import pandas_gbq as pdgbq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(
'C:/a/big-query--cd85abb654f9.json',
)

In [21]:
def run_BigQuery_table(sql):
    dotenv.load_dotenv()
    credentials, project = google.auth.default(
        scopes=[
            "https://www.googleapis.com/auth/cloud-platform",
            "https://www.googleapis.com/auth/drive",
            "https://www.googleapis.com/auth/bigquery",
        ]
    )
    pdgbq.context.credentials = credentials
    bigquery.Client(project, credentials)
    output = pd.read_gbq(sql, project_id=project, dialect='standard')
    return output

In [16]:
sql__my_bytes = """
select cast(string_field_0 as bytes) as mybytes
from my_schema.external_table__date_table
limit 1

"""
sql__my_bytes = """
select string_field_0 as mybytes
from my_schema.external_table__date_table
limit 1
"""

bq_output = run_BigQuery_table(sql__my_bytes)
# mybytes =  bq_output.mybytes[0] 
mybytes = bytes( bq_output.mybytes[0] , 'utf-8')

print(f"==== bytes = {mybytes}")

==== bytes = b'grass_date'


In [5]:
sql__my_key = """
select string_field_0 as mykey
from my_schema.external_table__date_table
limit 1
"""

bq_output = run_BigQuery_table(sql__my_key)
mykey = bq_output.mykey[0]
print(f"==== key = {mykey}")



==== key = grass_date


In [22]:
sql__decode = f"""
SELECT ANS.FUNCTION( KEYS.KEYSET_CHAIN('{mykey}', {mybytes}), columns_name, "" )
"""

bq_output = run_BigQuery_table(sql__decode)

data = pd.json_normalize( bq_output['f0_'][0] )

data

,kms_resource_name,first_level_keyset,second_level_keyset
0,grass_date,b'grass_date',None


In [99]:
import re

def replace_string(column_name, my_key, my_bytes):
    replaced = f"""AEAD.DECRYPT_STRING( KEYS.KEYSET_CHAIN( '{my_key}' , {my_bytes})
                                , from_base64({column_name})
                                , "" ) as decrypted_{column_name}"""
    return replaced

def change_sql(your_sql):
    string_need_to_replace = re.findall(f'{your_function}\(.*?\)', your_sql)
    for before_replace in string_need_to_replace:
        column_name = before_replace[before_replace.find("(")+1:before_replace.rfind(")")].strip()
        after_replace = replace_string(column_name, my_key, my_key)
        your_sql = your_sql.replace(before_replace, after_replace)
    return your_sql

#####################################################


your_sql = """ 
SELECT col1, col2, descrypt( col3), col4, descrypt(col5 ) , col6 , descrypt(col7)
FROM ...
"""
print(f"==== before \n{your_sql}")


your_function = 'descrypt'
my_key = 'testing_key'
my_bytes = b'mybytes'

print("\n\n")

print(f"==== after \n{change_sql(your_sql)}")


==== before 
 
SELECT col1, col2, descrypt( col3), col4, descrypt(col5 ) , col6 , descrypt(col7)
FROM ...




==== after 
 
SELECT col1, col2, AEAD.DECRYPT_STRING( KEYS.KEYSET_CHAIN( 'testing_key' , testing_key)
                                , from_base64(col3)
                                , "" ) as decrypted_col3, col4, AEAD.DECRYPT_STRING( KEYS.KEYSET_CHAIN( 'testing_key' , testing_key)
                                , from_base64(col5)
                                , "" ) as decrypted_col5 , col6 , AEAD.DECRYPT_STRING( KEYS.KEYSET_CHAIN( 'testing_key' , testing_key)
                                , from_base64(col7)
                                , "" ) as decrypted_col7
FROM ...

